In [ ]:
import pandas as pd
import numpy as np
import json
import io
import tifffile
import calendar
import time
from oauthlib.oauth2 import BackendApplicationClient
from requests_oauthlib import OAuth2Session

In [ ]:
with open("log.json") as e:
    creds = json.load(e)

CLIENT_ID = creds["client_id"]
CLIENT_SECRET = creds["client_secret"]
TOKEN_URL = 'https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token'

def get_authenticated_session():
    client = BackendApplicationClient(client_id=CLIENT_ID)
    oauth = OAuth2Session(client=client)
    token = oauth.fetch_token(
        token_url=TOKEN_URL,
        client_secret=CLIENT_SECRET,
        include_client_id=True
    )
    return oauth

oauth = get_authenticated_session()
print("Authentication successful.")

In [ ]:
csv_file = 'populated_bird_data.csv'  
df = pd.read_csv(csv_file)
print(f"Loaded {len(df)} locations to process.")

for col in ['ndvi', 'ndwi', 'ndmi', 'ndbi', 'mndwi']:
    df[col] = np.nan

In [ ]:
evalscript = """
function setup() {
  return {
    input: ["B03", "B04", "B08", "B11"],
    output: { bands: 5, sampleType: "FLOAT32" }
  };
}

function evaluatePixel(sample) {
  let ndvi = index(sample.B08, sample.B04); 
  let ndwi = index(sample.B03, sample.B08); 
  let ndmi = index(sample.B08, sample.B11); 
  let ndbi = index(sample.B11, sample.B08);
  let mndwi = index(sample.B03, sample.B11);
  return [ndvi, ndwi, ndmi, ndbi, mndwi];
}
"""

def fetch_indices_for_point(lat, lon, year, month, session):
    offset = 0.0001
    bbox = [lon - offset, lat - offset, lon + offset, lat + offset]
    
    last_day = calendar.monthrange(int(year), int(month))[1]
    start_date = f"{int(year)}-{int(month):02d}-01"
    end_date = f"{int(year)}-{int(month):02d}-{last_day}"
    json_request = {
        'input': {
            'bounds': {
                'bbox': bbox,
                'properties': { 'crs': 'http://www.opengis.net/def/crs/OGC/1.3/CRS84' }
            },
            'data': [{
                'type': 'S2L2A',
                'dataFilter': {
                    'timeRange': {
                        'from': f'{start_date}T00:00:00Z',
                        'to': f'{end_date}T23:59:59Z'
                    },
                    'mosaickingOrder': 'leastCC',
                },
            }]
        },
        'output': {
            'width': 1,  
            'height': 1,
            'responses': [{ 'identifier': 'default', 'format': { 'type': 'image/tiff' } }]
        },
        'evalscript': evalscript
    }

    url = "https://sh.dataspace.copernicus.eu/api/v1/process"
    
    try:
        response = session.post(url, json=json_request)
        
        if response.status_code == 401:
            print("\nToken expired. Refreshing...", end='')
            session = get_authenticated_session()
            response = session.post(url, json=json_request)
            
        if response.status_code != 200:
            return None, session 
            
        image_arr = tifffile.imread(io.BytesIO(response.content))
        
        if image_arr.ndim == 3:
            vals = image_arr.flatten()
            return vals, session
            
    except Exception as e:
        return None, session

    return None, session

In [ ]:
print("\nStarting processing...")
start_time = time.time()

for index, row in df.iterrows():
    if not np.isnan(row['ndvi']):
        continue
        
    print(f"Doing location {index} / {len(df)}...", end='\r')
    
    vals, oauth = fetch_indices_for_point(
        row['latitude'], 
        row['longitude'], 
        row['year'], 
        row['month'], 
        oauth
    )
    
    if vals is not None and len(vals) == 5:
        df.at[index, 'ndvi'] = vals[0]
        df.at[index, 'ndwi'] = vals[1]
        df.at[index, 'ndmi'] = vals[2]
        df.at[index, 'ndbi'] = vals[3]
        df.at[index, 'mndwi'] = vals[4] 
    
    if index % 100 == 0:
        df.to_csv(csv_file, index=False)

df.to_csv(csv_file, index=False)
print(f"\n\nDone! Processed {len(df)} rows in {time.time() - start_time:.0f} seconds.")
print(df.head())